# Model 3: Transformer Fine-Tuning

Transformer-based models leverage contextual embeddings learned from large-scale corpora, enabling superior semantic understanding compared to traditional neural architectures.

Steps:
1. Load raw or lightly cleaned text
2. Tokenization with pretrained tokenizer (attention masks)
3. Model loading and fine-tuning
4. Evaluation on test set
5. Save metrics

## 1. Load raw text

In [ ]:
from pathlib import Path
import re

import numpy as np
import pandas as pd

DATA_DIR = Path("../data")

train_df = pd.read_csv(DATA_DIR / "train.txt", sep=";", header=None, names=["text", "emotion"])
test_df = pd.read_csv(DATA_DIR / "test.txt", sep=";", header=None, names=["text", "emotion"])


def clean_text(text: str) -> str:
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text


train_df["clean_text"] = train_df["text"].apply(clean_text)
test_df["clean_text"] = test_df["text"].apply(clean_text)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

train_df.head()

## 2. Tokenization with pretrained tokenizer

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
max_length = 128

label_names = sorted(train_df["emotion"].unique())
label_to_id = {label: idx for idx, label in enumerate(label_names)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

train_df["label"] = train_df["emotion"].map(label_to_id)
test_df["label"] = test_df["emotion"].map(label_to_id)

train_dataset = Dataset.from_pandas(train_df[["clean_text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["clean_text", "label"]])

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_batch(batch):
    return tokenizer(
        batch["clean_text"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )

train_dataset = train_dataset.map(tokenize_batch, batched=True)
test_dataset = test_dataset.map(tokenize_batch, batched=True)

train_dataset = train_dataset.remove_columns(["clean_text"])
test_dataset = test_dataset.remove_columns(["clean_text"])
train_dataset.set_format("torch")
test_dataset.set_format("torch")

print("Labels:", label_names)

## 3. Load model and fine-tune

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_names),
    id2label=id_to_label,
    label2id=label_to_id,
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

training_args = TrainingArguments(
    output_dir="../results/transformer_checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer

## 4. Train and evaluate

In [ ]:
train_result = trainer.train()
metrics = trainer.evaluate()

print("Eval metrics:", metrics)

## 5. Save metrics

In [ ]:
from datetime import datetime

results_dir = Path("../results")
results_dir.mkdir(parents=True, exist_ok=True)
metrics_path = results_dir / "metrics.csv"

row = {
    "model": "TRANSFORMER",
    "accuracy": round(metrics.get("eval_accuracy", 0.0), 4),
    "precision": round(metrics.get("eval_precision", 0.0), 4),
    "recall": round(metrics.get("eval_recall", 0.0), 4),
    "f1": round(metrics.get("eval_f1", 0.0), 4),
    "timestamp": datetime.now().isoformat(timespec="seconds"),
}

metrics_df = pd.DataFrame([row])

if metrics_path.exists():
    metrics_df.to_csv(metrics_path, mode="a", header=False, index=False)
else:
    metrics_df.to_csv(metrics_path, index=False)

print(f"Saved metrics to: {metrics_path}")
metrics_df